# This is just a notebook to visualise 1kHz filtered raw data

In [1]:
cd "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/"

\\10.69.168.1\crnldata\waking\audrey_hay\L1imaging\AnalysedMarch2023


In [36]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
import fnmatch
import os
import mmap
import xarray as xr
import dask.array as da
%matplotlib widget
from IPython.display import display
from ipyfilechooser import FileChooser

from ephyviewer import mkQApp, MainViewer, TraceViewer

In [57]:
def other_mmap(filename, shape, dtype, chunks):
    with open(filename, "rb") as fh:
        # `mmap` duplicates the file descriptor
        # `0` means map the full file
        mm = mmap.mmap(fh.fileno(), 0, access=mmap.ACCESS_READ)

    # coerce to NumPy array of expected type and shape
    a = np.asarray(mm).view(dtype).reshape(shape)

    # Don't call `asarray`
    # name=False to avoid hashing
    a_d = da.from_array(a, chunks=chunks, asarray=False, name=False)
    return a_d

# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file

In [45]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in strore")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='C:\Users\Manip2\DATA\CamilleData\enregistrements\22.04\cheeseboard blue\trial 8.1', filename…

Stored 'dpath' (str)
Stored 'dpath' (str)


Load LFPs data

In [64]:
folderpath = Path(dpath)

if find_files_with_string(folderpath,  ".bin"): #Bonsai
    matching_file = find_files_with_string(folderpath, ".bin")
    All = np.fromfile(matching_file, dtype="int16")
    All = All.reshape(-1,32)
    print('File loaded: OE32channels.bin')
elif find_files_with_string(folderpath,  "continuous.dat"): #OpenEphys
    matching_file = find_files_with_string(folderpath, "continuous.dat")
    All = np.fromfile(matching_file, dtype="int16")
    All = All.reshape(-1,64)
    print('File loaded: continuous.dat')
elif find_files_with_string(folderpath,  "RawDataChannelExtractedDS.npy"): #OpenEphys Gaelle's Data
    matching_file = find_files_with_string(folderpath, "RawDataChannelExtractedDS.npy")
    All = np.load(matching_file, mmap_mode= 'r')
    All = All.reshape(-1,32)
    print('File loaded: RawDataChannelExtractedDS.npy')

File loaded: OE32channels.bin


Visualise All

In [65]:
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

win.add_view(view1)

#Run
win.show()
app.exec_()


0